# Imports

In [ ]:
import glob
import os
import shutil

#the following three imports have to be installed
from PIL import Image
import numpy as np
from tqdm import tqdm

import Utilities.affine_math_functions as amf
import Utilities.preprocessing_functions as pf

## Hyperparameters

In [ ]:
OUTSIZE = (105, 75)

SCALE_WIDTH_CM = 2. # width in cm between the two outer scale points
SCALE_PADDING_FACTOR = 1.4 # factor to widen the width of the box 
                           # in relation to the distance between the scale points.
    
                           # Without padding, the scale points show a distance of 2cm.
                           # However, the scales are drawn at least 0.4cm longer on each side.
                           # Due to padding, the box has a width of 2.8cm = 1.4*2cm.

NUM_SEGMENTS = 3     # An odd-numbered number of segments that determines which parts are being cut.
                     # The segments overlie one another to 50%.   

# Extra Augmentationen:
NUM_TRANSLATIONS = 9 # Number of random tranlations that are applied to each segment
TRANSLATION_DELTA = 30   # Translations are randomly selected out of the interval
                         # [-TRANSLATION_DELTA, TRANSLATION_DELTA]
                       
SCALE_PATH = '../all_scale_data.npy'
INFOLDER = '../DIP_images_fresh/all/'
OUTFOLDER = f"images/images_classification/"

## Check if all files and folder exist 

In [ ]:
#check if SCALE_PATH exists, if not raise an error to request the existance.
if not os.path.exists(SCALE_PATH):
    raise FileNotFoundError('The SCALE_PATH has to exist in order to start the preprocessing. Please create the file via get_scale_data.ipynb first or correct the SCALE_PATH.')

#check if the INFOLDER exists, if not: raise an error to request the existence of the folder
if not os.path.exists(INFOLDER):
    raise FileNotFoundError('The INFOLDER has to exist with all 666 images included in order to start the preprocessing.') 

#check if the OUTFOLDER exists, if not: create the directory
if not os.path.exists(OUTFOLDER):
    os.mkdir(OUTFOLDER)    


# Preprocess images

In [ ]:
scale_data = np.load(SCALE_PATH, allow_pickle = True)
scale_data = scale_data[()]
inpaths = glob.glob(INFOLDER + '*.jpg')

outratio = OUTSIZE[0]/OUTSIZE[1]       
n = len(inpaths)   
# Randomly chosen translations for x- and y-coordinates
translations = np.random.randint(low=-TRANSLATION_DELTA,high=TRANSLATION_DELTA, size = (n,NUM_TRANSLATIONS,2))
transformed_landmarks = {}

for i in tqdm(range(n, desc = "Preprocess images", unit= "images"):  
    inpath = inpaths[i]
    filename = inpath[len(INFOLDER):]
    img = Image.open(inpath)
    
    a1, m1 = pf.affine_rotation_from_scale_data(img.size, scale_data[filename])
    cropped, box = pf.crop_from_scale_affinity(img, a1, m1, scale_data[filename], SCALE_PADDING_FACTOR)
    a2, m2_local, reg_data = pf.affine_rotation_from_scale_data_crop(cropped)
    
    # Calculate the left_middle point of the shoot for segmentation
    #Therefore, get the scale points and replicate steps from above
    l = np.array(scale_data[filename]['left'])
    r = np.array(scale_data[filename]['right'])
    l[1] = img.height-l[1]
    r[1] = img.height-r[1] 
    
    sd = np.array([ [l[0], l[1]], [r[0], r[1]] ])
    sd_rotated = amf.affine_rotation(a1, m1, sd)
    lnew = sd_rotated[0,:]
    rnew = sd_rotated[1,:]
    # Calculate the distance 
    v = rnew - lnew
    # Widen the width
    lnew = lnew - (SCALE_PADDING_FACTOR - 1)/2*v
    
    # local coordinates in the box
    lnew = lnew - np.array(box[:2])
    
    # get local coordinate of the middle of the shoot with the regression data
    lnew[1] = reg_data[0][0]*lnew[0] + reg_data[1]
    
    # Transform to global rotated coordinate system
    lnew = lnew + np.array(box[:2])
    
    # Transform to coordinates in the global original system
    lnew = np.array(lnew).reshape((1,2))
    left_middle = amf.inverse_affine_rotation(a1,m1,lnew)
    
    #Add both angles
    angle = a1 + a2
    #Calculate relation of pixels to cm 
    box_width_in_cm = SCALE_PADDING_FACTOR * SCALE_WIDTH_CM
    cm_pixel_ratio = (box[2] - box[0]) / box_width_in_cm #x/2.8 = 1cm

    #get middle of the image
    middle_image = np.array(img.size) / 2
    
    #rotate the image
    img = img.rotate(angle)
    
    #get the wanted left_middle point
    left_middle = amf.affine_rotation(angle, middle_image, left_middle).flatten()
    
    # determine the width and height of a segment in cm 
    cm_width = SCALE_PADDING_FACTOR * SCALE_WIDTH_CM / ( (NUM_SEGMENTS-1) / 2 + 1 )
    cm_height = cm_width / outratio 
    
    #calculate the box for each segment
    for s in range(NUM_SEGMENTS):
    
        w_left = np.array([s*cm_pixel_ratio * cm_width / 2, -(cm_pixel_ratio*cm_height/2)])
        w_right = np.array([w_left[0] + cm_pixel_ratio * cm_width , -w_left[1]])

        left = left_middle + w_left 
        right = left_middle + w_right

        #get translations
        T = np.array(np.r_[np.matrix(((0,0))), translations[i,...]])
        
        #apply all translations on each segment
        for t in range(NUM_TRANSLATIONS+1):
            translation = T[t,:]

            left_t = left + translation
            right_t = right + translation

            box = (left_t[0],left_t[1],right_t[0],right_t[1]) #determine the final box

            cropped = img.crop(box) #crop the image

            cropped = cropped.resize(OUTSIZE) #resize the image

            filename_t = filename[:-len(".jpg")] + f"_s{s}_t{t}.jpg"
            cropped.save(OUTFOLDER + filename_t)